In [2]:
from math import log
import operator

In [5]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet) #数据集大小
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]   #获取分类标签
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1  #每个类中数据个数统计
    shannonEnt = 0.0
    for key in labelCounts:  #信息熵计算
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) 
    return shannonEnt
    
def splitDataSet(data,feature,value):
    #return splitted data
    newdata=[]
    for rows in data:
        if rows[feature]==value : 
            reduceFeatVec=rows[:feature]
            reduceFeatVec.extend(rows[feature+1:])
            newdata.append(reduceFeatVec)
    return newdata 
    
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1  #计算分类依据的个数
    baseEntropy = calcShannonEnt(dataSet)   #计算原始分类的信息熵
    bestInfoGain = 0.0; 
    bestFeature = -1
    for i in range(numFeatures):    #对apple进行分类
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:  #计算该种分类的信息熵
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy  #计算当前分类的信息增益
        if (infoGain > bestInfoGain):  #比较那种分类的信息增益最大并返回
            bestInfoGain = infoGain
            bestFeature = i    
    return bestFeature
def majority(labelValues):
    labelCount =dict()
    for label in labelValues:
        labelCount[label]=labelCount.setdefault(label,0)+1
    labelCountSort = sorted(labelCount.items(),key = lambda x:x[1],reverse=True)
    return labelCountSort[0][0]

    
def createTree(dataSet,features):
    labelValues = [data[-1] for data in dataSet]
    if labelValues.count(labelValues[0])==len(labelValues): #全部为同一类别
        return labelValues[0]
    if len(dataSet[0])==1: #特征为空
        return majority(labelValues)
    bestAxis = chooseBestFeatureToSplit(dataSet)#最优特征下标,信息增益     
    bestFeature = features[bestAxis]#最优特征值
    ID3Tree ={bestFeature:{}}#构造决策树
        
    del(features[bestAxis]) #对每一个子集，以｛全部特征-特征Ａ｝为特征

    featureValues = set([data[bestAxis] for data in dataSet])
    for key in featureValues:
        subFeatures = features[:]
        subDataSet = splitDataSet(dataSet,bestAxis,key)
        ID3Tree[bestFeature][key] = createTree(subDataSet,subFeatures)
    return ID3Tree
def preres(row,mytree,labels):
    res=-1
    firstStr=list(mytree.keys())[0]
    index=labels.index(firstStr)
    secondDict=mytree[firstStr]
    num_of_choice=len(list(secondDict.keys()))

    for i in range(num_of_choice):
        value=list(secondDict.keys())[i]
        if row[index]==value:
            if type(secondDict[value]).__name__=='dict':
                return preres(row,secondDict[value],labels)
            else:
                res=secondDict[value]
                return res
        else: continue
    
def calerrtr(data,mytree,labels): 
    err_train=0
    for i in range(len(data)):
        row=data[i]
        pre=preres(row,mytree,labels)
        if (pre==row[-1]):
            pass
        else:
            err_train+=1
    print(err_train/len(data))
    return err_train/len(data)



In [4]:
import numpy as np
import pandas as pd
import math
def createDataSet():
   
    label=[]
    for i in range(13):
        label.append("feature"+str(i+1))
    
    df=pd.read_csv(r'mcmost1_train.csv')
    #print(df)
    data_1=np.array(df)
    data_2=data_1.tolist()
    data_all=[i[1:] for i in data_2]
    total_num=len(data_all)
    data_train=data_all[0:math.floor(0.8*total_num)]
    data_test=data_all[math.floor(0.8*total_num):]
    return data_train,data_test,label

In [71]:
data_train,data_test,labels=createDataSet()
data=data_train
labelstmp=[]
for i in range (len(labels)):
    labelstmp.append(labels[i])
mytree=createTree(data,labelstmp)
#print(mytree)
calerrtr(data_train,mytree,labels)
calerrtr(data_test,mytree,labels)

In [12]:
def createTreepru1(dataSet,features,depth,d=1):
    labelValues = [data[-1] for data in dataSet]
    if labelValues.count(labelValues[0])==len(labelValues): #全部为同一类别
        return labelValues[0]
    if len(dataSet[0])==1 or d>=depth: #特征为空
        return majority(labelValues)
        
    bestAxis = chooseBestFeatureToSplit(dataSet)#最优特征下标,信息增益     
    bestFeature = features[bestAxis]#最优特征值
    ID3Tree ={bestFeature:{}}#构造决策树
        
    del(features[bestAxis]) #对每一个子集，以｛全部特征-特征Ａ｝为特征

    featureValues = set([data[bestAxis] for data in dataSet])
    for key in featureValues:
        subFeatures = features[:]
        subDataSet = splitDataSet(dataSet,bestAxis,key)
        ID3Tree[bestFeature][key] = createTreepru1(subDataSet,subFeatures,depth,d+1)
    return ID3Tree


def createTreepru2(dataSet,features,samplesize):
    labelValues = [data[-1] for data in dataSet]
    if labelValues.count(labelValues[0])==len(labelValues): #全部为同一类别
        return labelValues[0]
    if len(dataSet[0])==1 or len(dataSet)<=samplesize: #特征为空
        return majority(labelValues)
        
    bestAxis = chooseBestFeatureToSplit(dataSet)#最优特征下标,信息增益     
    bestFeature = features[bestAxis]#最优特征值
    ID3Tree ={bestFeature:{}}#构造决策树
        
    del(features[bestAxis]) #对每一个子集，以｛全部特征-特征Ａ｝为特征

    featureValues = set([data[bestAxis] for data in dataSet])
    for key in featureValues:
        subFeatures = features[:]
        subDataSet = splitDataSet(dataSet,bestAxis,key)
        ID3Tree[bestFeature][key] = createTreepru2(subDataSet,subFeatures,samplesize)
    return ID3Tree

In [63]:
data_train,data_test,labels=createDataSet()
data=data_train
labelstmp=[]
for i in range (len(labels)):
    labelstmp.append(labels[i])
mytree=createTreepru1(data,labelstmp,2)
print(mytree)

{'feature13': {1.0: 2.0, 2.0: 2.0, 3.0: 2.0, 4.0: 2.0, 5.0: 2.0, 6.0: 2.0, 7.0: 2.0}}


In [64]:
calerrtr(data_test,mytree,labels)

0.18055555555555555


0.18055555555555555

In [28]:
#TIC
import pandas as pd
import numpy as np

def loadDataset(infile):
    df = pd.read_csv(infile, sep='\t', header=None, dtype=str, na_filter=False)
    #df_file=pd.DataFrame(np.array(df).astype(np.float),columns=features)
    #return df_file
    return np.array(df).astype(np.float)

a=loadDataset("ticdata2000.txt")
a=a.tolist()
data=a

label=[]
for i in range(86):
    label.append("feature"+str(i+1))
    
labelstmp=[]
for i in range (len(label)):
    labelstmp.append(label[i])
res=rankFeatureToSplit(data)

In [27]:
def rankFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1  #计算分类依据的个数
    baseEntropy = calcShannonEnt(dataSet)   #计算原始分类的信息熵
    bestInfoGain = 0.0; 
    rankFeature = dict()
    for i in range(numFeatures):    #对apple进行分类
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:  #计算该种分类的信息熵
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy  #计算当前分类的信息增益
        rankFeature[i]=rankFeature.setdefault(i,0)+infoGain
    return rankFeature

In [29]:
print(res)

{0: 0.015260720872135203, 1: 0.0007972822836823656, 2: 0.001287692808422225, 3: 0.0005673619228074855, 4: 0.010969412972420545, 5: 0.001325673540907235, 6: 0.002717666510158423, 7: 0.0014978297648832184, 8: 0.0026618466803074448, 9: 0.004403065917262972, 10: 0.0013805581447980941, 11: 0.004094660202578981, 12: 0.0025870964022050025, 13: 0.0007919611356045908, 14: 0.0015812407664169692, 15: 0.0051803347071330275, 16: 0.0022013867339356064, 17: 0.006831164233656373, 18: 0.004038144610563732, 19: 0.0005314337561095894, 20: 0.003038820977239731, 21: 0.002817611106230633, 22: 0.0038777631096795218, 23: 0.003055046551533658, 24: 0.0057033267101205265, 25: 0.002258365457761924, 26: 0.0006301586559178673, 27: 0.003965423638379428, 28: 0.003843912099470992, 29: 0.0058723926584536845, 30: 0.00580995979980925, 31: 0.005004381154667004, 32: 0.0001843989563080073, 33: 0.0061478583256497354, 34: 0.0032251182946089196, 35: 0.0030691580471602364, 36: 0.007133877723624893, 37: 0.0007340288912277559, 38

In [ ]:
from operator import itemgetter
sorted(res.items(), key=itemgetter(1), reverse=True)[0:22]

# ffff

### hhkjk